In [1]:
# import os
# import cv2
# import albumentations as A
# from tqdm import tqdm

# # 설정
# txt_base_dir = '/content/drive/MyDrive/01/Datasets/final-airplane--1' #로컬에 맞춰서 개선선
# input_img_dir = os.path.join(txt_base_dir, 'images')
# input_lbl_dir = os.path.join(txt_base_dir, 'labels')
# output_img_dir = input_img_dir
# output_lbl_dir = input_lbl_dir

# # 증강 정의
# transform = A.Compose([
#     A.RandomSizedCrop(min_max_height=(300, 500), size=(640, 640), p=0.3),
#     A.Rotate(limit=30, p=0.4),
#     A.HorizontalFlip(p=0.5),
#     A.RandomRotate90(p=0.2),
#     A.HueSaturationValue(p=0.4),
#     A.RGBShift(p=0.3),
#     A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.1), contrast_limit=0.3, p=0.4),
#     A.Resize(height=640, width=640, p=1.0),
#     A.GaussianBlur(p=0.1)
# ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))


# # train_iter_01 ~ train_iter_10
# for i in range(1, 11):
#     iter_id = f"{i:02d}"
#     txt_name = f"train_iter_{iter_id}.txt"
#     txt_path = os.path.join(txt_base_dir, txt_name)

#     if not os.path.exists(txt_path):
#         print(f"❌ {txt_name} 없음 - 건너뜀")
#         continue

#     with open(txt_path, 'r') as f:
#         image_paths = [line.strip() for line in f.readlines()]

#     print(f"🔄 {txt_name} 처리 중 ({len(image_paths)}장)")

#     new_lines = []

#     for img_path in tqdm(image_paths):
#         img_name = os.path.basename(img_path)
#         img_stem = os.path.splitext(img_name)[0]
#         input_img_path = os.path.join(input_img_dir, img_name)
#         input_lbl_path = os.path.join(input_lbl_dir, f"{img_stem}.txt")

#         image = cv2.imread(input_img_path)
#         if image is None:
#             print(f"❌ 이미지 로드 실패: {input_img_path}")
#             continue

#         # 라벨 로드
#         bboxes = []
#         class_labels = []
#         if os.path.exists(input_lbl_path):
#             with open(input_lbl_path, 'r') as f:
#                 for line in f:
#                     parts = line.strip().split()
#                     if len(parts) == 5:
#                         cls, x, y, w, h = parts
#                         bboxes.append([float(x), float(y), float(w), float(h)])
#                         class_labels.append(int(cls))

#         try:
#             transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
#         except Exception as e:
#             print(f"⚠️ bbox 오류로 해당 이미지 건너뜀: {img_path}")
#             continue

#         aug_image = transformed['image']
#         aug_bboxes = transformed['bboxes']
#         aug_labels = transformed['class_labels']

#         if not aug_bboxes:
#             continue

#         # 저장 파일명
#         aug_img_name = f"aug_{img_name}"
#         aug_lbl_name = f"aug_{img_stem}.txt"
#         save_img_path = os.path.join(output_img_dir, aug_img_name)
#         save_lbl_path = os.path.join(output_lbl_dir, aug_lbl_name)

#         # 저장
#         cv2.imwrite(save_img_path, aug_image)
#         with open(save_lbl_path, 'w') as f:
#             for bbox, cls in zip(aug_bboxes, aug_labels):
#                 f.write(f"{cls} {' '.join(f'{x:.6f}' for x in bbox)}\n")

#         # txt에 기록할 새로운 이미지 경로
#         new_lines.append(save_img_path)

#     # 기존 train_iter_xx.txt에 덧붙이기
#     with open(txt_path, 'a') as f:
#         for new_line in new_lines:
#             f.write(f"{new_line}\n")

#     print(f"✅ {txt_name} 에 증강 이미지 {len(new_lines)}장 추가 완료")

# print("🎉 전체 train_iter_xx 파일에 증강 이미지 경로 추가 완료")
#이건 전체과정 한번에 다한거임

In [2]:
from competition_utils import *

# 제출 함수를 리스트로 정의 (submission_N_학번)
# 깃허브에는 최대 3개 업로드 데모시 하이퍼파라미터 튜닝 후 1개만 제출
submission_functions = ['submission_1_20214173', 'submission_2_20214173', 'submission_3_20214173']  
for submission_function in submission_functions:
    exec(f"from {submission_function} import {submission_function}")

# 분석 방향에 따라 iteration 수 수정
# 평가시에는 데모와 다른 스플릿 시드를 만들어 [1, 10]로 진행 예정
iterations = [1, 10] 

# 'Crown Detection'는 예제 데이터셋, 데모 및 평가시에는 'CV_Competition'
Dataset_Name = 'Crown Detection' 

# 결과를 모아 하나의 csv 파일에 저장
results_df = pd.DataFrame(columns=[
    'Experiment Time', 'Iteration', 'Submission Function', 
    'IoU', 'Dice', 'Precision', 'Recall', 'Output Json Path',
])
csv_filename = f"Evaluation_Results_{datetime.now().strftime('%y%m%d_%H%M%S')}.csv"

# 서로 다른 iteration, submission function으로 실험 진행
for iteration in range(iterations[0], iterations[1]+1):
    yaml_path = f'Datasets/{Dataset_Name}/data_iter_{iteration:02d}.yaml'
    for submission_function in submission_functions:
        ex_time = datetime.now().strftime('%y%m%d_%H%M%S')
        output_json_path = f"{ex_time}_{submission_function}_Iter_{iteration}_detection_results.json"
        globals()[submission_function](yaml_path, output_json_path)
        labels_dir = f"Datasets/{Dataset_Name}/labels"  
        vis_output_dir = f"{ex_time}_visualization_results"  
        image_level_result_path =f"{ex_time}_{submission_function}_Iter_{iteration}_image_level_results.json"
        stats = eval_and_vis(yaml_path, output_json_path, labels_dir, image_level_result_path, vis_output_dir, vis=False) # 분석 방향에 따라 vis=True 설정
        new_row = {
            'Experiment Time': ex_time,
            'Iteration': iteration,
            'Submission Function': submission_function,
            'IoU': stats['IoU']['avg'],
            'Dice': stats['Dice']['avg'],
            'Precision': stats['Precision']['avg'],
            'Recall': stats['Recall']['avg'],
            'Output Json Path': output_json_path,
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        results_df.to_csv(csv_filename, index=False)

engine\trainer: task=250423_023526_Train, mode=train, model=yolov8n.pt, data=Datasets/Crown Detection/data_iter_01.yaml, epochs=20, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=tmp, name=250423_023526_yolov8n_Crown Detection_Iter_1, exist_ok=True, pretrained=False, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# 분석 방향에 따라 전체 결과를 모아 평균-표준편차-통계테스트 결과 저장 

keep_columns = ['Iteration','Submission Function']
keep_measures = ['IoU','Dice','Precision','Recall']  

reduction = 'Iteration'
row = 'Measure Type'
column = 'Submission Function'
reference_column = 'submission_1_2023110214'

custom_fmt_template = '{mean_fmt} ± {std_fmt} {significance}'
significance_levels = [0.1, 0.05, 0.01]
decimal_places = 3
transpose = True
make_tables(csv_filename, keep_columns, keep_measures, reduction, row, column,  
            reference_column, custom_fmt_template, significance_levels, decimal_places, transpose)

CSV 파일로 저장 완료: Evaluation_Results_250418_081601_agg.csv
